# Notes
Maybe I should add the condition $c_0 + d_1 = a_0$ to the equations - this might make it easier to weed out non-possible solutions.

In [3]:
# Imports
import sympy as sp
import numpy as np

import matplotlib.pyplot as plt
import itertools
import time
import os

In [4]:
n = sp.Symbol("n")
n

n

In [5]:
t = sp.Symbol("t")
t

t

In [6]:
r = sp.Rational

# Helper functions

In [7]:
def create_poly_from_coeffs(coeffs, var):
    poly = coeffs[0]
    for i, coeff in enumerate(coeffs[1:]):
        poly += coeff * var**(i+1)
    return sp.expand(poly)

In [8]:
create_poly_from_coeffs([1,2,3],n)

3*n**2 + 2*n + 1

In [9]:
def create_poly_from_roots(roots, var):
    poly = 1
    for root in roots:
        poly *= (n - root)
    return poly

In [10]:
create_poly_from_roots([1,2,3],n)

(n - 3)*(n - 2)*(n - 1)

In [11]:
def create_poly_from_quad_factors(coeffs, var):
    poly = 1; coeffs = coeffs[::-1]
    start_idx = 0
    if len(coeffs) % 3 == 1:
        poly *= (var - (coeffs[0]))
        start_idx = 1
    if len(coeffs) % 3 == 2:
        poly *= (var - (coeffs[0])) * (var - coeffs[1])
        start_idx = 2
        
    for i in range(start_idx, len(coeffs), 3):
        poly *= coeffs[i] * var**2 + coeffs[i+1] * var + coeffs[i+2]
        
    return poly

In [12]:
create_poly_from_quad_factors([1,2,3,1,2,4],n)

(3*n**2 + 2*n + 1)*(4*n**2 + 2*n + 1)

In [1]:
def calc_frac(a,b,k=0,num_iter=10):
    return a(k) + b(k+1)/calc_frac(a,b,k+1,num_iter-1) if num_iter else a(k)

In [2]:
a = lambda k: 1
b = lambda k: 1
calc_frac(a,b,0,1_000) # 6.180339887 4989484820 4586834365 6381177203 0917980576 2862135448 6227052604 6281890244

1.618033988749895

# dn/cn finder
We have:
\begin{align*}
 &c_n + d_{n+1} = s_na_n (\heartsuit) \\ 
 &c_nd_n = s_ns_{n-1}b_n (\clubsuit)
\end{align*}
From $(\heartsuit)$ we can derive that it's most likely that $\deg(d_n) = \deg(c_n) = \deg(s_n) + \deg(a_n)$. Thus we must have, from $(\clubsuit)$ that $2(\deg(s_n) + \deg(a_n)) = 2\deg(s_n) + \deg(b_n)$ i.e. $2\deg(a_n) = \deg(b_n)$.

In [37]:
# def dncn_finder(a_coeffs, b_coeffs, deg_s=0, ret_first=True): 
#     # Polynomial degrees
#     deg_a = len(a_coeffs) - 1
#     deg_b = len(b_coeffs) - 1
#     deg_c = deg_d = deg_s + deg_a
    
#     # Polynomial coefficients
#     a_coeffs = a_coeffs[::-1]
#     b_coeffs = b_coeffs[::-1]
#     c_coeffs = [sp.Symbol(f"gamma_{k}") for k in range(0, deg_c + 1)]
#     d_coeffs = [sp.Symbol(f"delta_{k}") for k in range(0, deg_d + 1)]
#     s_coeffs = [sp.Symbol(f"sigma_{k}") for k in range(0, deg_s + 1)]  
    
#     # Define polynomials
#     an = create_poly_from_coeffs(a_coeffs, var=n)
#     bn = create_poly_from_coeffs(b_coeffs, var=n)
#     sn = create_poly_from_coeffs(s_coeffs, var=n)
# #     cn = create_poly_from_coeffs(c_coeffs, var=n)
# #     dn = create_poly_from_coeffs(d_coeffs, var=n)
    
#     sn_minus_1 = sp.expand(sn.subs({n:n-1}))
# #     dn_plus_1 = sp.expand(dn.subs({n:n+1}))
    
#     # Setup equations and solve - note, the number of variables = deg(d) + deg(c) + deg(s)
#     num_vars = deg_d + deg_c + deg_s # note that the parity of num_vars is decided by the parity of deg_s
#     eqs = []
    
#     heart = sp.Eq(cn + dn_plus_1, sn * an)
#     clubs = sp.Eq(cn*dn, -bn*sn*sn_minus_1)
#     for i in range(round(num_vars/2)):
#         n0 = i - round(num_vars/2)
#         eqs.append(heart.subs({n:n0}))
#         eqs.append(clubs.subs({n:n0}))
        
#     sol = sp.solve(eqs)
#     return sol

In [39]:
dncn_finder([3,7,3], [-2,-1,6,0,0], deg_s=2)

# With combinatorics
Only shown to work for for deg <= 2, but the speed is insane!

In [25]:
def dncn_finder_coeffs(a_coeffs, b_coeffs, deg_s=0, ret_first=True): 
    # Polynomial degrees
    deg_a = len(a_coeffs) - 1
    deg_b = len(b_coeffs) - 1
    deg_c = deg_d = deg_s + deg_a
    
    # Polynomial coefficients
    a_coeffs = a_coeffs[::-1]
    b_coeffs = b_coeffs[::-1]
    s_coeffs = [sp.Symbol(f"sigma_{k}") for k in range(1, deg_s + 1)]  + [1]
    
    # Define polynomials
    an = create_poly_from_coeffs(a_coeffs, var=n)
    bn = create_poly_from_coeffs(b_coeffs, var=n)
    sn = create_poly_from_coeffs(s_coeffs, var=n)
        
    sn_minus_1 = sp.expand(sn.subs({n:n-1}))
    
    leading_coeff, factors_rhs = sp.factor_list(-sn*sn_minus_1*bn)    
    factors = []
    
    for pr in factors_rhs:
        for i in range(pr[1]):
            factors.append(pr[0])
    possible_d_roots = list(itertools.chain(*list([itertools.combinations(factors, i) for i in range(int(deg_d/2),deg_d + 1)])))
    possible_d_roots = [l for l in possible_d_roots if sum([sp.degree(p, n) for p in l]) == deg_d]
    possible_c_roots = []
    for d_roots in possible_d_roots:
        l = factors.copy()
        for d_root in d_roots:
            l.remove(d_root)
        possible_c_roots.append(l)
    
    sols = []
    
    lead_d = sp.Symbol("lambda")
    start_time = time.time()
    for i, (c_roots,d_roots) in enumerate(zip(possible_c_roots, possible_d_roots)):
        print(f"[{i+1}/{len(possible_c_roots)}] --- Time: {time.time() - start_time} s")
        c_poly = 1; d_poly = 1
        for c_root,d_root in zip(c_roots, d_roots):
            c_poly *= c_root
            d_poly *= d_root
        
        lhs_coeffs = sp.Poly((leading_coeff/lead_d) * c_poly + lead_d * d_poly.subs({n:n+1}), n).coeffs()
        rhs_coeffs = sp.Poly(sp.expand(sn * an), n).coeffs()
        
        eqs = [sp.Eq(lhs, rhs, evaluate=False) for lhs, rhs in zip(lhs_coeffs, rhs_coeffs)]
        sol = sp.solve(eqs)
        if sol:
            print(f"Solution found at [{i+1}]")
            sols.append({"sol": sol, "lead_d": lead_d, "lead": leading_coeff, 
                         "cn": sp.factor(c_poly), "dn": leading_coeff * sp.factor(d_poly),
                         "sn": sp.factor(sn)})
            if ret_first:
                break 
    
    return sols

In [26]:
dncn_finder_coeffs([3,7,4], [-2, -4, -2, 0, 0])

[1/6] --- Time: 0.0 s
[2/6] --- Time: 0.13364005088806152 s
Solution found at [2]


[{'sol': [{lambda: 2}],
  'lead_d': lambda,
  'lead': 2,
  'cn': n*(n + 1),
  'dn': 2*n*(n + 1),
  'sn': 1}]

In [208]:
dncn_finder_coeffs([3,7,3], [-2,-1,6,0,0], deg_s=2, ret_first=False)

[1/14] --- Time: 0.0 s
[2/14] --- Time: 0.0576169490814209 s
[3/14] --- Time: 0.11047577857971191 s
Solution found at [3]
[4/14] --- Time: 0.17609047889709473 s
[5/14] --- Time: 0.25278711318969727 s
[6/14] --- Time: 0.3309049606323242 s
[7/14] --- Time: 0.37485623359680176 s
[8/14] --- Time: 0.420665979385376 s
[9/14] --- Time: 0.49393153190612793 s
[10/14] --- Time: 0.579204797744751 s
[11/14] --- Time: 0.640150785446167 s
[12/14] --- Time: 0.7108070850372314 s
[13/14] --- Time: 0.7770137786865234 s
[14/14] --- Time: 0.8418388366699219 s


[{'sol': [{lambda: 1, sigma_1: -4, sigma_2: 7}],
  'lead_d': lambda,
  'lead': 1,
  'cn': n**2*(n**2 + n*sigma_2 - 2*n + sigma_1 - sigma_2 + 1),
  'dn': (n + 2)*(2*n - 3)*(n**2 + n*sigma_2 + sigma_1),
  'sn': n**2 + n*sigma_2 + sigma_1}]

In [209]:
dncn_finder_coeffs([3,3,1], [-2,3,0,0,0], deg_s=2)

[1/14] --- Time: 0.0 s
[2/14] --- Time: 0.08656978607177734 s
[3/14] --- Time: 0.1873011589050293 s
Solution found at [3]


[{'sol': [{lambda: 1, sigma_1: -2, sigma_2: 3}],
  'lead_d': lambda,
  'lead': 1,
  'cn': n**2*(n**2 + n*sigma_2 - 2*n + sigma_1 - sigma_2 + 1),
  'dn': n*(2*n - 3)*(n**2 + n*sigma_2 + sigma_1),
  'sn': n**2 + n*sigma_2 + sigma_1}]

In [210]:
dncn_finder_coeffs([6,9,5,1], [-1, 0, 0, 0, 0, 0, 0], deg_s=8)

[]

In [182]:
sp.expand((2*n + 1)*(3*n*(n+1) + 1))

6*n**3 + 9*n**2 + 5*n + 1

In [211]:
dncn_finder_coeffs([10,15,9,2], [-16, 0, 0, 0, 0, 0, 0], deg_s=5)

[1/40] --- Time: 0.0 s
[2/40] --- Time: 0.15827465057373047 s
[3/40] --- Time: 0.275928258895874 s
[4/40] --- Time: 0.33644723892211914 s
[5/40] --- Time: 0.40514612197875977 s
[6/40] --- Time: 0.46692919731140137 s
[7/40] --- Time: 0.5367431640625 s
[8/40] --- Time: 0.6035802364349365 s
[9/40] --- Time: 0.6865890026092529 s
[10/40] --- Time: 0.7546737194061279 s
[11/40] --- Time: 0.8394465446472168 s
[12/40] --- Time: 0.9066808223724365 s
[13/40] --- Time: 0.9884600639343262 s
[14/40] --- Time: 1.0505118370056152 s
[15/40] --- Time: 1.1342885494232178 s
[16/40] --- Time: 1.2011404037475586 s
[17/40] --- Time: 1.2779028415679932 s
[18/40] --- Time: 1.3487138748168945 s
[19/40] --- Time: 1.4195237159729004 s
[20/40] --- Time: 1.4754071235656738 s
[21/40] --- Time: 1.5531656742095947 s
[22/40] --- Time: 1.6170296669006348 s
[23/40] --- Time: 1.6938221454620361 s
[24/40] --- Time: 1.7476825714111328 s
[25/40] --- Time: 1.811474084854126 s
[26/40] --- Time: 1.8822851181030273 s
[27/40] ---

[]

In [192]:
sp.expand((2*n + 1)*(5*n*(n+1) + 2))

10*n**3 + 15*n**2 + 9*n + 2

# In terms of roots

In [13]:
def dncn_finder_roots(a_coeffs, b_coeffs, deg_s=0, ret_first=True): 
    # Polynomial degrees
    deg_a = len(a_coeffs) - 1
    deg_b = len(b_coeffs) - 1
    deg_c = deg_d = deg_s + deg_a
    
    # Polynomial coefficients
    a_coeffs = a_coeffs[::-1]
    b_coeffs = b_coeffs[::-1]
    s_roots = [sp.Symbol(f"sigma_{k}") for k in range(1, deg_s + 1)]
    s_lead = 1
    for sr in s_roots:
        s_lead *= (1/sr)
    
    # Define polynomials
    an = create_poly_from_coeffs(a_coeffs, var=n)
    bn = create_poly_from_coeffs(b_coeffs, var=n)
    sn = s_lead * create_poly_from_roots(s_roots, var=n)

    sn_minus_1 = sn.subs({n:n-1})
    
    leading_coeff, factors_rhs = sp.factor_list(sp.Poly(-sn*sn_minus_1*bn))    
    factors = []
    
    for pr in factors_rhs:
        for i in range(pr[1]):
            factors.append(pr[0])
    possible_d_roots = list(itertools.chain(*list([itertools.combinations(factors, i) for i in range(int(deg_d/2),deg_d + 1)])))
    possible_d_roots = [l for l in possible_d_roots if sum([sp.degree(p, n) for p in l]) == deg_d]
    possible_c_roots = []
    for d_roots in possible_d_roots:
        l = factors.copy()
        for d_root in d_roots:
            l.remove(d_root)
        possible_c_roots.append(l)
    
    sols = []
    
    lead_d = sp.Symbol("lambda")
    start_time = time.time()
    for i, (c_roots,d_roots) in enumerate(zip(possible_c_roots, possible_d_roots)):
        print(f"[{i+1}/{len(possible_c_roots)}] --- Time since start: {time.time() - start_time} s --- Estimated time to finish: {time.time() - (time.time() - start_time) * len(possible_c_roots)/(i+1)}")
        c_poly = 1; d_poly = 1
        for c_root,d_root in zip(c_roots, d_roots):
            c_poly *= c_root
            d_poly *= d_root
        
        lhs_coeffs = sp.Poly((leading_coeff/lead_d) * c_poly + lead_d * d_poly.subs({n:n+1}), n).coeffs()
        rhs_coeffs = sp.Poly(sp.expand(sn * an), n).coeffs()
        
        eqs = [sp.Eq(lhs, rhs, evaluate=False) for lhs, rhs in zip(lhs_coeffs, rhs_coeffs)]
        sol = sp.solve(eqs)
        if sol:
            print(f"Solution found at [{i+1}]")
            sols.append({"sol": sol, "lead_d": lead_d, "lead": leading_coeff, 
                         "cn": sp.factor(c_poly), "dn": sp.factor(d_poly),
                         "sn": sp.factor(sn)})
            if ret_first:
                break 
    
    return sols

In [21]:
sp.expand(-2*n**2 * (n + 1)**2)

-2*n**4 - 4*n**3 - 2*n**2

In [276]:
dncn_finder_roots([3,3,1], [-2,3,0,0,0], deg_s=2)

[1/70] --- Time since start: 0.0 s --- Estimated time to finish: 0.0
[2/70] --- Time since start: 9.097695350646973 s --- Estimated time to finish: 318.41933727264404
[3/70] --- Time since start: 13.30426836013794 s --- Estimated time to finish: 310.43292840321857
[4/70] --- Time since start: 21.290303468704224 s --- Estimated time to finish: 372.5803107023239
[5/70] --- Time since start: 22.216824054718018 s --- Estimated time to finish: 311.03553676605225
[6/70] --- Time since start: 22.976791381835938 s --- Estimated time to finish: 268.06256612141925
[7/70] --- Time since start: 23.3597674369812 s --- Estimated time to finish: 233.597674369812
[8/70] --- Time since start: 23.657968521118164 s --- Estimated time to finish: 207.00722455978394
[9/70] --- Time since start: 23.783632516860962 s --- Estimated time to finish: 184.98380846447415
[10/70] --- Time since start: 23.912289142608643 s --- Estimated time to finish: 167.3860239982605
[11/70] --- Time since start: 24.31820249557495

[{'sol': [{lambda: 1, sigma_1: -3/2 + sqrt(17)/2, sigma_2: -sqrt(17)/2 - 3/2},
   {lambda: 1, sigma_1: -sqrt(17)/2 - 3/2, sigma_2: -3/2 + sqrt(17)/2}],
  'lead_d': lambda,
  'lead': 1,
  'cn': Poly(n**4*(1/sigma_1)*(1/sigma_2) - 2*n**3*(1/sigma_1)*(1/sigma_2) - n**3*(1/sigma_1) - n**3*(1/sigma_2) + n**2*(1/sigma_1)*(1/sigma_2) + n**2*(1/sigma_1) + n**2*(1/sigma_2) + n**2, n, 1/sigma_1, 1/sigma_2, domain='ZZ'),
  'dn': Poly(2*n**4*(1/sigma_1)*(1/sigma_2) - 3*n**3*(1/sigma_1)*(1/sigma_2) - 2*n**3*(1/sigma_1) - 2*n**3*(1/sigma_2) + 3*n**2*(1/sigma_1) + 3*n**2*(1/sigma_2) + 2*n**2 - 3*n, n, 1/sigma_1, 1/sigma_2, domain='ZZ'),
  'sn': (n - sigma_1)*(n - sigma_2)/(sigma_1*sigma_2)}]

In [1]:
# dncn_finder_roots([10,15,9,2], [-16, 0, 0, 0, 0, 0, 0], deg_s=3)

In [2]:
# dncn_finder_roots([7,7,1], [8,0,0,0,0], deg_s=3)

# In terms of quadratic factors

In [14]:
def dncn_finder_quads(a_coeffs, b_coeffs, deg_s=0, ret_first=True, start_idx=0): 
    # Polynomial degrees
    deg_a = len(a_coeffs) - 1
    deg_b = len(b_coeffs) - 1
    deg_c = deg_d = deg_s + deg_a
    
    # Polynomial coefficients
    a_coeffs = a_coeffs[::-1]
    b_coeffs = b_coeffs[::-1]
    s_coeffs = [sp.Symbol(f"sigma_{k}") for k in range(0, deg_s + 1)]
    
    # Define polynomials
    an = create_poly_from_coeffs(a_coeffs, var=n)
    bn = create_poly_from_coeffs(b_coeffs, var=n)
    sn = create_poly_from_quad_factors(s_coeffs, var=n)

    sn_minus_1 = sn.subs({n:n-1})
    
    leading_coeff, factors_rhs = sp.factor_list(sp.Poly(-sn*sn_minus_1*bn))    
    factors = []
    
    for pr in factors_rhs:
        for i in range(pr[1]):
            factors.append(pr[0])
    possible_d_roots = list(itertools.chain(*list([itertools.combinations(factors, i) for i in range(int(deg_d/2),deg_d + 1)])))
    possible_d_roots = [l for l in possible_d_roots if sum([sp.degree(p, n) for p in l]) == deg_d]
    possible_c_roots = []
    for d_roots in possible_d_roots:
        l = factors.copy()
        for d_root in d_roots:
            l.remove(d_root)
        possible_c_roots.append(l)
    
    sols = []
    
    lead_d = sp.Symbol("lambda")
    start_time = time.time()
    for i, (c_roots,d_roots) in enumerate(zip(possible_c_roots, possible_d_roots)):
        if (i + 1) < start_idx:
            continue
        print(f"[{i+1}/{len(possible_c_roots)}] --- Time since start: {time.time() - start_time} s --- Estimated time to finish: {(time.time() - start_time) * len(possible_c_roots)/(i+1)}")
        c_poly = 1; d_poly = 1
        for c_root,d_root in zip(c_roots, d_roots):
            c_poly *= c_root
            d_poly *= d_root
        
        lhs_coeffs = sp.Poly((leading_coeff/lead_d) * c_poly + lead_d * d_poly.subs({n:n+1}), n).coeffs()
        rhs_coeffs = sp.Poly(sp.expand(sn * an), n).coeffs()
        
        eq_sn = sp.Eq(sn.subs({n:0}), 1, evaluate=False)
        eqs = [eq_sn] + [sp.Eq(lhs, rhs, evaluate=False) for lhs, rhs in zip(lhs_coeffs, rhs_coeffs)]
        sol = sp.solve(eqs)
        if sol:
            print(f"Solution found at [{i+1}]")
            sols.append({"sol": sol, "lead_d": lead_d, "lead": leading_coeff, 
                         "cn": sp.factor(c_poly), "dn": sp.factor(d_poly),
                         "sn": sp.factor(sn)})
            if ret_first:
                break 
    
    return sols

In [288]:
dncn_finder_quads([3,3,1], [-2,3,0,0,0], deg_s=3)

[1/52] --- Time since start: 0.0 s --- Estimated time to finish: 0.0
[2/52] --- Time since start: 0.24335169792175293 s --- Estimated time to finish: 6.327144145965576
[3/52] --- Time since start: 0.4059140682220459 s --- Estimated time to finish: 7.035843849182129
[4/52] --- Time since start: 0.5700178146362305 s --- Estimated time to finish: 7.410231590270996
[5/52] --- Time since start: 0.7535269260406494 s --- Estimated time to finish: 7.8366800308227536
[6/52] --- Time since start: 0.8373012542724609 s --- Estimated time to finish: 7.256610870361328
[7/52] --- Time since start: 0.9011311531066895 s --- Estimated time to finish: 6.694117137363979
[8/52] --- Time since start: 2.2933218479156494 s --- Estimated time to finish: 14.906592011451721
[9/52] --- Time since start: 2.457882881164551 s --- Estimated time to finish: 14.20110109117296
Solution found at [9]


[{'sol': [{lambda: 2, sigma_0: -2/3, sigma_1: 1, sigma_2: 1/3, sigma_3: 3/2}],
  'lead_d': lambda,
  'lead': 1,
  'cn': Poly(2*n**5*sigma_2 + 2*n**4*sigma_1 - 7*n**4*sigma_2 + 2*n**3*sigma_0 - 5*n**3*sigma_1 + 8*n**3*sigma_2 - 3*n**2*sigma_0 + 3*n**2*sigma_1 - 3*n**2*sigma_2, n, sigma_0, sigma_1, sigma_2, sigma_3, domain='ZZ'),
  'dn': Poly(n**5*sigma_2 + n**4*sigma_1 - 2*n**4*sigma_2*sigma_3 - n**4*sigma_2 + n**3*sigma_0 - 2*n**3*sigma_1*sigma_3 - n**3*sigma_1 + n**3*sigma_2*sigma_3**2 + n**3*sigma_2*sigma_3 - 2*n**2*sigma_0*sigma_3 - n**2*sigma_0 + n**2*sigma_1*sigma_3**2 + n**2*sigma_1*sigma_3 + n*sigma_0*sigma_3**2 + n*sigma_0*sigma_3, n, sigma_0, sigma_1, sigma_2, sigma_3, domain='ZZ'),
  'sn': (n - sigma_3)*(n**2*sigma_2 + n*sigma_1 + sigma_0)}]

In [ ]:
dncn_finder_quads([7,7,2], [8,0,0,0,0], deg_s=4, start_idx=29)

[19/196] --- Time since start: 0.0 s --- Estimated time to finish: 0.0
[20/196] --- Time since start: 4346.40472984314 s --- Estimated time to finish: 42594.76635246277
[21/196] --- Time since start: 8840.862074613571 s --- Estimated time to finish: 82514.71269639333
[22/196] --- Time since start: 42136.60661029816 s --- Estimated time to finish: 375398.8588917472
[23/196] --- Time since start: 46663.177047252655 s --- Estimated time to finish: 397651.4217939791
[24/196] --- Time since start: 46728.37675142288 s --- Estimated time to finish: 381615.07680328685
[25/196] --- Time since start: 46793.23678898811 s --- Estimated time to finish: 366858.9764256668
[26/196] --- Time since start: 46857.59877753258 s --- Estimated time to finish: 353234.20616909175
[27/196] --- Time since start: 46921.3722345829 s --- Estimated time to finish: 340614.40585104626
[28/196] --- Time since start: 46986.06425023079 s --- Estimated time to finish: 328902.4497516155
[29/196] --- Time since start: 47044

In [302]:
# dncn_finder_quads([3,9,7], [-2, -3, 3, 7, 3], deg_s=3)

[1/52] --- Time since start: 0.0 s --- Estimated time to finish: 0.0
[2/52] --- Time since start: 0.0987403392791748 s --- Estimated time to finish: 2.567248821258545
[3/52] --- Time since start: 0.21941256523132324 s --- Estimated time to finish: 3.8031511306762695
[4/52] --- Time since start: 0.31412816047668457 s --- Estimated time to finish: 4.083666086196899
[5/52] --- Time since start: 0.4258272647857666 s --- Estimated time to finish: 4.428603553771973
[6/52] --- Time since start: 0.513636589050293 s --- Estimated time to finish: 4.451517105102539
[7/52] --- Time since start: 0.5913846492767334 s --- Estimated time to finish: 4.393143108912876
[8/52] --- Time since start: 37.163434743881226 s --- Estimated time to finish: 241.56232583522797
[9/52] --- Time since start: 41.3651921749115 s --- Estimated time to finish: 238.99888812171088
[10/52] --- Time since start: 73.20500302314758 s --- Estimated time to finish: 380.66601572036745
[11/52] --- Time since start: 78.2515001296997

[]

In [306]:
dncn_finder_quads([3,7,3], [-2, -11, -19, -13, -3], deg_s=3)

[1/52] --- Time since start: 0.0 s --- Estimated time to finish: 0.0
[2/52] --- Time since start: 0.21047329902648926 s --- Estimated time to finish: 5.472305774688721
[3/52] --- Time since start: 0.3869667053222656 s --- Estimated time to finish: 6.7074228922526045
[4/52] --- Time since start: 0.5196120738983154 s --- Estimated time to finish: 6.754956960678101
[5/52] --- Time since start: 0.6851687431335449 s --- Estimated time to finish: 7.136126899719239
[6/52] --- Time since start: 0.8128595352172852 s --- Estimated time to finish: 7.044782638549805
[7/52] --- Time since start: 0.8866281509399414 s --- Estimated time to finish: 6.586380549839565
[8/52] --- Time since start: 39.76245403289795 s --- Estimated time to finish: 258.45595121383667
[9/52] --- Time since start: 43.89639401435852 s --- Estimated time to finish: 253.62360986073813
[10/52] --- Time since start: 64.60302686691284 s --- Estimated time to finish: 335.93573970794677
[11/52] --- Time since start: 67.9809539318084

[]